### This is a single time process to merge all the information from multiple sources around lyric processing:

- lyric cleaning
- sentimental analyze
- themes
- embeddings

Everything above is examined and integrated

In [ ]:
max_week_csv = "max_weeks_0223.csv" # include unique songs and their max weeks on the chart
lyrics_embedding_js = "song_lyrics_embeddings_26221.json"
lyrics_theme_js = "song_themes_26221.json"
lyrics_js = "cleaned_lyrics.json"
lyrics_sentiment_csv = "lyrics_sentiment_scores.csv"

In [ ]:
import pandas as pd
import json

### Billboard data

In [ ]:
billboard_df = pd.read_csv(max_week_csv)
# include only ["date", "song", "artist", "peak-rank", "max-weeks-on-board"]
billboard_df = billboard_df[["date", "song", "artist", "peak-rank", "max-weeks-on-board"]]

In [ ]:
billboard_df.head()

,date,song,artist,peak-rank,max-weeks-on-board
0,1990-08-11,"""B"" Girls",Young And Restless,54,15.0
1,1973-05-19,"""Cherry Cherry"" from Hot August Night",Neil Diamond,31,10.0
2,1973-12-08,"""Having A Party"" Medley",The Ovations (Featuring Louis Williams),56,9.0
3,1974-02-16,"""Joy"" Pt. I",Isaac Hayes,30,9.0
4,1977-04-16,"""Roots"" Medley",Quincy Jones,57,7.0


In [ ]:
billboard_df[billboard_df["song"] == "Glory"]

,date,song,artist,peak-rank,max-weeks-on-board
8671,2015-03-14,Glory,Common & John Legend,49,3.0


### Lyrics


In [ ]:
with open(lyrics_js, 'r') as json_file:
    all_song_lyrics_dict = json.load(json_file)

lyrics_df = pd.DataFrame(list(all_song_lyrics_dict.items()),columns = ['song - artist','lyrics'])

In [ ]:
lyrics_df.head()

,song - artist,lyrics
0,Glory - Common & John Legend,\n\nOne day when the glory comes\nIt will be o...
1,Glory And Gore - Lorde,\n\nThere's a humming in the restless summer a...
2,Glory Bound - The Grass Roots,All the sun on the outside\nDoesn't touch what...
3,Glory Days - Bruce Springsteen,\n\nOh yeah\nC'mon\nWooh\nHuh\n\nI had a frien...
4,Glory Glory - The Rascals,"I saw a child, he was carryin' a light\nI aske..."


### Lyric embedding

In [ ]:
def read_from_json(file_name):
    # Create an empty dictionary to store the embeddings
    result_dict = {}
    
    # Open the JSON file and read each line
    with open(file_name, 'r') as infile:
        for line in infile:
            # Parse the JSON object from the line
            # song-artist is the key, and the embedding/theme is the value
            song_info = json.loads(line)
            
            # Update the embeddings dictionary with the song and its embedding
            result_dict.update(song_info)
    
    return result_dict

In [ ]:
embedding_dict = read_from_json(lyrics_embedding_js)


In [ ]:
embedding_df = pd.DataFrame(list(embedding_dict.items()),columns = ['song - artist','embedding'])

embedding_df.head()

,song - artist,embedding
0,Glory Glory - The Rascals,"[0.019659634679555893, -0.020099421963095665, ..."
1,Glory And Gore - Lorde,"[0.025850359350442886, 0.033453404903411865, -..."
2,Glory Bound - The Grass Roots,"[0.018391937017440796, -0.024728847667574883, ..."
3,Gnarly - Kodak Black Featuring Lil Pump,"[0.019683968275785446, -0.033210866153240204, ..."
4,"Glory Of Love (Theme From ""The Karate Kid Part...","[-0.014439592137932777, -0.022376852110028267,..."


### Lyric themes

In [ ]:
theme_dict = read_from_json(lyrics_theme_js)

theme_df = pd.DataFrame(list(theme_dict.items()),columns = ['song - artist','themes'])

theme_df.head()


,song - artist,themes
0,Go Part 1 - Polo G Featuring G Herbo,"Struggle, Violence, Wealth"
1,Glow - Drake Featuring Kanye West,"Celebration, Friendship, Individuality, Love, ..."
2,Go For Soda - Kim Mitchell,"Celebration, Love"
3,Gnarly - Kodak Black Featuring Lil Pump,"Adventure, Celebration, Friendship, Individual..."
4,"Go Loko - YG, Tyga & Jon Z","Love, Sexuality, Individuality"


In [ ]:
theme_df[theme_df["song - artist"] == "Glory - Common & John Legend"]

,song - artist,themes
19,Glory - Common & John Legend,"Struggle, Politics, Community, History"


### Lyrics sentiment scores

In [ ]:
sentiment_df = pd.read_csv(lyrics_sentiment_csv)

In [ ]:
sentiment_df.head()

,song - artist,lyrics,average_positive,average_negative
0,Glory - Common & John Legend,one day when the glory comes it will be ours...,0.491604,0.036153
1,Glory And Gore - Lorde,there's a humming in the restless summer air...,0.337827,0.004051
2,Glory Bound - The Grass Roots,all the sun on the outside doesn't touch what ...,0.985848,0.000000
3,Glory Days - Bruce Springsteen,oh yeah c'mon wooh huh i had a friend was a...,0.493888,0.006323
4,Glory Glory - The Rascals,"i saw a child, he was carryin' a light i asked...",0.987065,0.000000


## merging ~~

Notice that we can merge everything with the `song - artist` columns

The max_week billboard data frame should have most data rows; however, during each processing, there are error cases so not all songs will have fully analyazed data:
- When scraping lyrics, about 1170 songs failed to obtain lyrics, and among those with lyrics avaliable in during request, about 15-20% are garbage/wrong data
- There are cleaning and preprocessing before the embedding/LLM/sentiment analyses; around 2.6k songs should have full avaliability.

In [ ]:
# Define a function to concatenate values of two columns
def concatenate_values(row):
    return str(row['song']) + " - " + str(row['artist'])

# Apply the function row-wise and store the result in a new column
billboard_df['song - artist'] = billboard_df.apply(lambda row: concatenate_values(row), axis=1)

In [ ]:
billboard_df.head()

,date,song,artist,peak-rank,max-weeks-on-board,song - artist
0,1990-08-11,"""B"" Girls",Young And Restless,54,15.0,"""B"" Girls - Young And Restless"
1,1973-05-19,"""Cherry Cherry"" from Hot August Night",Neil Diamond,31,10.0,"""Cherry Cherry"" from Hot August Night - Neil D..."
2,1973-12-08,"""Having A Party"" Medley",The Ovations (Featuring Louis Williams),56,9.0,"""Having A Party"" Medley - The Ovations (Featur..."
3,1974-02-16,"""Joy"" Pt. I",Isaac Hayes,30,9.0,"""Joy"" Pt. I - Isaac Hayes"
4,1977-04-16,"""Roots"" Medley",Quincy Jones,57,7.0,"""Roots"" Medley - Quincy Jones"


In [ ]:
# merge all other dataframes with billboard_df on the song - artist column
# This should be a left join to keep all data billboard_df

# merge lyrics_df
merged_df = pd.merge(billboard_df, lyrics_df, on='song - artist', how='left')

# merge embedding_df
merged_df = pd.merge(merged_df, embedding_df, on='song - artist', how='left')

# merge theme_df
merged_df = pd.merge(merged_df, theme_df, on='song - artist', how='left')

# merge sentiment_df
merged_df = pd.merge(merged_df, sentiment_df, on='song - artist', how='left')


In [ ]:
merged_df.head()

,date,song,artist,peak-rank,max-weeks-on-board,song - artist,lyrics_x,embedding,themes,lyrics_y,average_positive,average_negative
0,1990-08-11,"""B"" Girls",Young And Restless,54,15.0,"""B"" Girls - Young And Restless","\n\nHey, Slim. Come here, check this out\nUh-u...","[0.046063587069511414, -0.027632707729935646, ...","Adventure, Wealth, Individuality, Violence","hey, slim. come here, check this out uh-uh, ...",0.000000,0.004921
1,1973-05-19,"""Cherry Cherry"" from Hot August Night",Neil Diamond,31,10.0,"""Cherry Cherry"" from Hot August Night - Neil D...",Lyrics not available,NaN,NaN,lyrics not available,0.000000,0.000331
2,1973-12-08,"""Having A Party"" Medley",The Ovations (Featuring Louis Williams),56,9.0,"""Having A Party"" Medley - The Ovations (Featur...",We are approaching the five-year mark since th...,NaN,NaN,we are approaching the five-year mark since th...,0.000000,0.249287
3,1974-02-16,"""Joy"" Pt. I",Isaac Hayes,30,9.0,"""Joy"" Pt. I - Isaac Hayes","AAlex Turner - ""Stuck on the Puzzle""\nAlvvays ...",NaN,NaN,"aalex turner - ""stuck on the puzzle"" alvvays -...",0.708964,0.000000
4,1977-04-16,"""Roots"" Medley",Quincy Jones,57,7.0,"""Roots"" Medley - Quincy Jones","The Mabbot street entrance of nighttown, befor...",NaN,NaN,"the mabbot street entrance of nighttown, befor...",0.456224,0.002376


We can see this is *messy* as expected

In [ ]:
# drop the song - artist column
merged_df = merged_df.drop(columns=["song - artist"])

# drop lyrics_y (which is the lower cased lyric with all \n removed), use lyrics_x

merged_df = merged_df.drop(columns=["lyrics_y"])
merged_df = merged_df.rename(columns={"lyrics_x": "lyrics"})

# create a new column called "fully_available"
# when lyrics == "Lyrics not available", or any NaN -> fully_available is False

def is_fully_available(row):
    if row['lyrics'] == "Lyrics not available" or row.isnull().values.any():

        return False
    return True

merged_df['fully_available'] = merged_df.apply(lambda row: is_fully_available(row), axis=1)



In [ ]:
merged_df.head()

,date,song,artist,peak-rank,max-weeks-on-board,lyrics,embedding,themes,average_positive,average_negative,fully_available
0,1990-08-11,"""B"" Girls",Young And Restless,54,15.0,"\n\nHey, Slim. Come here, check this out\nUh-u...","[0.046063587069511414, -0.027632707729935646, ...","Adventure, Wealth, Individuality, Violence",0.000000,0.004921,True
1,1973-05-19,"""Cherry Cherry"" from Hot August Night",Neil Diamond,31,10.0,Lyrics not available,NaN,NaN,0.000000,0.000331,False
2,1973-12-08,"""Having A Party"" Medley",The Ovations (Featuring Louis Williams),56,9.0,We are approaching the five-year mark since th...,NaN,NaN,0.000000,0.249287,False
3,1974-02-16,"""Joy"" Pt. I",Isaac Hayes,30,9.0,"AAlex Turner - ""Stuck on the Puzzle""\nAlvvays ...",NaN,NaN,0.708964,0.000000,False
4,1977-04-16,"""Roots"" Medley",Quincy Jones,57,7.0,"The Mabbot street entrance of nighttown, befor...",NaN,NaN,0.456224,0.002376,False


In [ ]:
# save the merged dataframe to a csv file

merged_df.to_csv("merged_data.csv", index=False)